In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

last_scrape = '2023-11-27'

In [2]:
playURL = 'https://api-web.nhle.com/v1/gamecenter/2023020350/play-by-play'

In [3]:
plays = requests.get(playURL).json()
plays = plays['plays']

plays = pd.DataFrame(plays)

plays



for index, row in plays.iterrows():
    plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))

plays['second'] = plays['second'].astype(int)

first_plays = plays.loc[plays['period'] == 1]

first_plays

,eventId,period,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details,second
0,102,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,520,period-start,8,NaN,0
1,101,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,502,faceoff,11,"{'eventOwnerTeamId': 53, 'losingPlayerId': 847...",0
2,103,1,"{'number': 1, 'periodType': 'REG'}",00:13,19:47,1551,left,504,giveaway,12,"{'xCoord': 13, 'yCoord': -27, 'zoneCode': 'N',...",13
3,54,1,"{'number': 1, 'periodType': 'REG'}",00:14,19:46,1551,left,506,shot-on-goal,13,"{'xCoord': 56, 'yCoord': 36, 'zoneCode': 'O', ...",14
4,8,1,"{'number': 1, 'periodType': 'REG'}",00:15,19:45,1551,left,516,stoppage,14,{'reason': 'goalie-stopped-after-sog'},15
...,...,...,...,...,...,...,...,...,...,...,...,...
88,481,1,"{'number': 1, 'periodType': 'REG'}",19:31,00:29,1541,left,505,goal,274,"{'xCoord': -31, 'yCoord': 7, 'zoneCode': 'O', ...",1171
89,127,1,"{'number': 1, 'periodType': 'REG'}",19:31,00:29,1551,left,502,faceoff,278,"{'eventOwnerTeamId': 21, 'losingPlayerId': 848...",1171
90,34,1,"{'number': 1, 'periodType': 'REG'}",19:49,00:11,1551,left,516,stoppage,279,{'reason': 'offside'},1189
91,128,1,"{'number': 1, 'periodType': 'REG'}",19:49,00:11,1551,left,502,faceoff,281,"{'eventOwnerTeamId': 21, 'losingPlayerId': 847...",1189


In [5]:
def extract_values(row, key):
    return row.get(key)

keys_to_extract1 = ['periodType']

for key in keys_to_extract1:
    plays[key] = plays['periodDescriptor'].apply(lambda x: extract_values(x, key))
    
keys_to_extract_faceoff = ['eventOwnerTeamId', 'losingPlayerId', 'winningPlayerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_takeaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_shot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'awaySOG', 'homeSOG']
keys_to_extract_hit = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'hittingPlayerId', 'hitteePlayerId']
keys_to_extract_missedshot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'reason']
keys_to_extract_block = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'blockingPlayerId', 'shootingPlayerId']
keys_to_extract_giveaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_stoppage = ['reason']
keys_to_extract_goal = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'scoringPlayerId', 'scoringPlayerTotal', 'assist1PlayerId', 'assist1PlayerTotal', 'assist2PlayerId', 'assist2PlayerTotal', 'goalieInNetId', 'awayScore', 'homeScore']
keys_to_extract_penalty = ['eventOwnerTeamId', 'committedByPlayerId', 'xCoord', 'yCoord', 'zoneCode', 'typeCode', 'descKey', 'duration', 'drawnByPlayerId']

keys_lists = [
    keys_to_extract_faceoff,
    keys_to_extract_takeaway,
    keys_to_extract_shot,
    keys_to_extract_hit,
    keys_to_extract_missedshot,
    keys_to_extract_block,
    keys_to_extract_giveaway,
    keys_to_extract_stoppage,
    keys_to_extract_goal,
    keys_to_extract_penalty
]

plays

,eventId,period,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details,second,periodType
0,102,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,520,period-start,8,NaN,0,REG
1,101,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,502,faceoff,11,"{'eventOwnerTeamId': 53, 'losingPlayerId': 847...",0,REG
2,103,1,"{'number': 1, 'periodType': 'REG'}",00:13,19:47,1551,left,504,giveaway,12,"{'xCoord': 13, 'yCoord': -27, 'zoneCode': 'N',...",13,REG
3,54,1,"{'number': 1, 'periodType': 'REG'}",00:14,19:46,1551,left,506,shot-on-goal,13,"{'xCoord': 56, 'yCoord': 36, 'zoneCode': 'O', ...",14,REG
4,8,1,"{'number': 1, 'periodType': 'REG'}",00:15,19:45,1551,left,516,stoppage,14,{'reason': 'goalie-stopped-after-sog'},15,REG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,1265,4,"{'number': 4, 'periodType': 'OT'}",04:27,00:33,1341,right,506,shot-on-goal,856,"{'xCoord': -58, 'yCoord': -17, 'zoneCode': 'O'...",267,OT
284,1267,4,"{'number': 4, 'periodType': 'OT'}",04:36,00:24,1441,right,506,shot-on-goal,858,"{'xCoord': -83, 'yCoord': -1, 'zoneCode': 'O',...",276,OT
285,225,4,"{'number': 4, 'periodType': 'OT'}",04:39,00:21,1441,right,505,goal,859,"{'xCoord': -82, 'yCoord': 0, 'zoneCode': 'O', ...",279,OT
286,862,4,"{'number': 4, 'periodType': 'OT'}",04:39,00:21,1441,right,521,period-end,860,NaN,279,OT


In [6]:
def extract_values(row, keys):
    if isinstance(row, dict):
        return {key: row.get(key) for key in keys}
    else:
        return {key: None for key in keys}

for keys_to_extract in keys_lists:
    # Apply the function to each row
    extracted_df = pd.DataFrame(plays['details'].apply(lambda x: extract_values(x, keys_to_extract)).tolist())
    plays = pd.concat([plays, extracted_df], axis=1)
    
plays = plays.groupby(plays.columns, axis=1).first()
plays = plays.drop(columns=['details', 'periodDescriptor'])

In [7]:
plays.columns.tolist()
convert_columns = ['assist1PlayerId', 'assist2PlayerId', 'blockingPlayerId', 'committedByPlayerId', 'drawnByPlayerId', 'goalieInNetId', 'hitteePlayerId', 'hittingPlayerId', 'losingPlayerId', 'playerId', 'scoringPlayerId', 'shootingPlayerId', 'winningPlayerId']

plays[convert_columns] = plays[convert_columns].astype(pd.Int64Dtype(), errors='ignore')

convert_columns2 = ['assist1PlayerTotal', 'assist2PlayerTotal', 'awaySOG', 'awayScore', 'homeSOG', 'homeScore', 'scoringPlayerTotal', 'eventOwnerTeamId', 'duration']
plays[convert_columns2] = plays[convert_columns2].astype(pd.Int64Dtype(), errors='ignore')


In [8]:
plays_1 = plays.loc[plays['period'] == 1]
plays_2 = plays.loc[plays['period'] == 2]
plays_3 = plays.loc[plays['period'] == 3]
plays_4 = plays.loc[plays['period'] == 4]

In [22]:
# plays.loc[(plays['typeDescKey'] == 'shot-on-goal') | (plays_1['typeDescKey'] == 'penalty') | (plays_1['typeDescKey'] == 'goal')]

plays.loc[(plays['typeDescKey'] == 'shot-on-goal') & (plays['eventOwnerTeamId'] == 53)]['homeSOG']



3       1
9       2
15      3
27      4
31      5
32      6
52      7
63      8
75      9
85     10
97     11
154    12
163    13
165    14
166    15
167    16
180    17
181    18
182    19
188    20
192    21
193    22
276    23
279    24
280    25
281    26
282    27
283    28
284    29
Name: homeSOG, dtype: Int64

In [11]:
plays.columns.to_list()

['assist1PlayerId',
 'assist1PlayerTotal',
 'assist2PlayerId',
 'assist2PlayerTotal',
 'awaySOG',
 'awayScore',
 'blockingPlayerId',
 'committedByPlayerId',
 'descKey',
 'drawnByPlayerId',
 'duration',
 'eventId',
 'eventOwnerTeamId',
 'goalieInNetId',
 'hitteePlayerId',
 'hittingPlayerId',
 'homeSOG',
 'homeScore',
 'homeTeamDefendingSide',
 'losingPlayerId',
 'period',
 'periodType',
 'playerId',
 'reason',
 'scoringPlayerId',
 'scoringPlayerTotal',
 'second',
 'shootingPlayerId',
 'shotType',
 'situationCode',
 'sortOrder',
 'timeInPeriod',
 'timeRemaining',
 'typeCode',
 'typeDescKey',
 'winningPlayerId',
 'xCoord',
 'yCoord',
 'zoneCode']

In [20]:
plays.loc[(plays['typeDescKey'] == 'goal') & (plays['eventOwnerTeamId'] == 53)]

,assist1PlayerId,assist1PlayerTotal,assist2PlayerId,assist2PlayerTotal,awaySOG,awayScore,blockingPlayerId,committedByPlayerId,descKey,drawnByPlayerId,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
102,8477070,4,8482655,9,<NA>,1,<NA>,<NA>,None,<NA>,...,1551,324,03:03,16:57,505,goal,<NA>,-80.0,0.0,O
175,8477021,8,8479343,12,<NA>,2,<NA>,<NA>,None,<NA>,...,1451,493,16:22,03:38,505,goal,<NA>,-70.0,1.0,O
197,8479442,2,8480891,3,<NA>,2,<NA>,<NA>,None,<NA>,...,1551,560,01:08,18:52,505,goal,<NA>,87.0,-5.0,O
285,<NA>,<NA>,<NA>,<NA>,<NA>,3,<NA>,<NA>,None,<NA>,...,1441,859,04:39,00:21,505,goal,<NA>,-82.0,0.0,O
